# unstack과 pivot을 사용해 데이터를 넓은 포맷으로 리셰이핑
- 타이디한 데이터를 그렇지 않은 구조로 변환해야할 때도 있음
- unstack과 pivot은 긴 포맷에서 넓은 포맷으로 데이터를 리셰이핑하는데 유용함
- unstack과 stack이 정반대 / pivot과 melt가 반대의 일을 함

### 데이터 NLS
- 근무주 수와 대학등록 데이터 사용
- 쌓기/녹이기를 수행한 후 unstack과 pivot으로 원상복구하기

In [9]:
import pandas as pd

nls97= pd.read_csv('data/nls97f.csv')
nls97.set_index('originalid', inplace=True)

### stack 연산으로 데이터 쌓기

In [10]:
weeksworkedcols = ['weeksworked00', 'weeksworked01', 'weeksworked02', 'weeksworked03', 'weeksworked04']

In [11]:
weeksworkedstacked = nls97[weeksworkedcols].stack(dropna=False)

In [13]:
weeksworkedstacked.loc[[1, 2]]

originalid               
1           weeksworked00    53.0
            weeksworked01    52.0
            weeksworked02     NaN
            weeksworked03    42.0
            weeksworked04    52.0
2           weeksworked00    51.0
            weeksworked01    52.0
            weeksworked02    44.0
            weeksworked03    45.0
            weeksworked04    52.0
dtype: float64

### melt로 데이터 녹이기

In [17]:
weeksworkedmelted = nls97.reset_index().loc[:, ['originalid']+weeksworkedcols].melt(id_vars=['originalid'],
                                                               value_vars=weeksworkedcols,
                                                               var_name='year',
                                                               value_name='weeksworked')

In [19]:
weeksworkedmelted.loc[weeksworkedmelted.originalid.isin([1, 2])].sort_values(['originalid', 'year'])

,originalid,year,weeksworked
377,1,weeksworked00,53.0
9361,1,weeksworked01,52.0
18345,1,weeksworked02,NaN
27329,1,weeksworked03,42.0
36313,1,weeksworked04,52.0
8980,2,weeksworked00,51.0
17964,2,weeksworked01,52.0
26948,2,weeksworked02,44.0
35932,2,weeksworked03,45.0
44916,2,weeksworked04,52.0


### 길게 쌓은 데이터를 unstack 사용하여, 넓은 포맷으로 변환

In [20]:
weeksworked = weeksworkedstacked.unstack()

In [22]:
weeksworked.loc[[1,2]]

,weeksworked00,weeksworked01,weeksworked02,weeksworked03,weeksworked04
originalid,,,,,
1,53.0,52.0,NaN,42.0,52.0
2,51.0,52.0,44.0,45.0,52.0


### 길게 녹인 데이터를 pivot을 사용해 넓은 포맷으로 변환
- 열 이름의 뒷부분으로 사용할 열(year)과 되돌릴 열(여기서는 weeksworked)을 pivot 인자로 전달해야함
- 다중 수준의 열 이름이 반환되므로, 두번째 수준에서 이를 끄집어내어 수정한다 : [col[1] for col in weeksworked.columns[1:]]

In [26]:
weeksworked = weeksworkedmelted.pivot(index='originalid', columns='year', values=['weeksworked']).reset_index()

In [31]:
weeksworked.columns = ['originalid'] + [col[1] for col in weeksworked.columns[1:]]

In [33]:
weeksworked.loc[weeksworked.originalid.isin([1,2])].T

,0,1
originalid,1.0,2.0
weeksworked00,53.0,51.0
weeksworked01,52.0,52.0
weeksworked02,NaN,44.0
weeksworked03,42.0,45.0
weeksworked04,52.0,52.0
